- root_dir: the base of the BIDS directory.
- file_pattern: the wildcarded matching pattern for the base file. 
- output_file_name: the name of your output file. 

In [ ]:
root_dir = r"/Volumes/One Touch/BWH_PD_DBS_MixedTarget/vtas"
file_pattern = r"stimulation/*nii_generated_nifti.nii"
output_file_name = r"_sim-binary_model-simbio_hemi-bl_preprocessed"

In [ ]:
import os 
import subprocess
import numpy as np
from tqdm import tqdm
from glob import glob
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from calvin_utils.file_utils.import_functions import GiiNiiFileImport

def threshold_niftis_in_bids(root_dir=root_dir, file_pattern=file_pattern, output_file_name=output_file_name, threshold=0.05):
    '''
    Function to iterate over each subject in a BIDS directory, import them, threshold, and save the thresholded nifti file. 
    
    Will by default threshold everything to a value over 0.2 and binarize. 
    '''

    for sub_dir in tqdm(os.listdir(root_dir)):
        sub_dir_path = os.path.join(root_dir, sub_dir)
        if not os.path.isdir(sub_dir_path):
            continue
        
        # recursively glob all intermediate directories to the file pattern target.
        files1 = glob(root_dir+"/"+sub_dir+"**/"+file_pattern)

        # If identified, begin processing
        if files1:
            # Extract identified files
            file1 = files1[0]
            try:
                # Import the nifti and manipulate it
                importer = GiiNiiFileImport(os.path.dirname(file1), file_pattern=os.path.basename(file1))
                imports = importer.run()
                imports["thresholded"] = np.where(imports.iloc[:,0]>threshold, 1, 0)
                
                # Prepare Output File
                output_dir = os.path.dirname(file1)
                output_file_with_subject = os.path.basename(sub_dir_path)+"_"+output_file_name
                nifti_from_matrix(matrix=imports["thresholded"].to_numpy(), output_file=output_dir, output_name=output_file_with_subject, reference="MNI")
                
                print(f"\n ***Processed: {sub_dir_path}")
                
            except subprocess.CalledProcessError as e:
                print(f"Error processing {sub_dir_path}: {e}")
        else:
            print(f"Missing file(s) in {sub_dir_path}")

In [ ]:
threshold_niftis_in_bids(root_dir=root_dir, file_pattern=file_pattern, output_file_name=output_file_name, threshold=0.2)